In [1]:
import sqlite3
import pandas as pd
conn=sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = conn.cursor()

In [2]:
qry="select * from sqlite_master"
pd.read_sql(qry,conn)


,type,name,tbl_name,rootpage,sql
0,table,Bookings,Bookings,2,"CREATE TABLE ""Bookings"" (\n ""bookid"" int(4) N..."
1,index,sqlite_autoindex_Bookings_1,Bookings,3,None
2,table,Facilities,Facilities,50,"CREATE TABLE ""Facilities"" (\n ""facid"" int(1) ..."
3,index,sqlite_autoindex_Facilities_1,Facilities,51,None
4,table,Members,Members,52,"CREATE TABLE ""Members"" (\n ""memid"" int(2) NOT..."
5,index,sqlite_autoindex_Members_1,Members,53,None


In [3]:
#
df = pd.read_sql_query("SELECT * FROM Bookings ", conn)

# Print head of DataFrame
print(df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [4]:
sum(df.slots)


9191

In [5]:
df1 = pd.read_sql_query("SELECT * FROM Facilities ", conn)
# Print head of DataFrame
print(df1.head())


   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [6]:
df2 = pd.read_sql_query("select * from Members ", conn)
# Print head of DataFrame
print(df2.head())


   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  



### /* Q10: Produce a list of facilities with a total revenue less than 1000.
### The output of facility name and total revenue, sorted by revenue. Remember
### that there's a different cost for guests and members! */


In [7]:
q='''
SELECT query.facility,SUM(cost) as revenue
FROM (SELECT f.name AS facility,
        CASE WHEN b.memid=0 THEN f.guestcost* b.slots ELSE f.membercost* b.slots END AS cost
FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid=f.facid) AS query
GROUP BY query.facility
HAVING revenue<1000
ORDER BY revenue
'''
pd.read_sql_query(q,conn)


,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270



### /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */



In [8]:
q1='''
select m.firstname || m.surname as member,q.firstname ||q.surname as recommender
from Members as m inner join Members as q on q.memid=m.recommendedby
where q.memid!=0 and q.recommendedby is not null
order by q.surname,q.firstname

'''
pd.read_sql_query(q1,conn)

,member,recommender
0,RamnareshSarwin,FlorenceBader
1,JoanCoplin,TimothyBaker
2,MatthewGenting,GeraldButters
3,TimothyBaker,JemimaFarrell
4,DavidPinker,JemimaFarrell
5,HenriettaRumney,MatthewGenting
6,DouglasJones,DavidJones
7,NancyDare,JaniceJoplette
8,DavidJones,JaniceJoplette
9,JohnHunt,MillicentPurview



### /* Q12: Find the facilities with their usage by member, but not guests */


In [9]:
q2='''
select f.name as facility,sum(slots) as usage
from Bookings as b inner join Facilities as f on b.facid=f.facid
where b.memid !=0
group by facility
order by usage
'''
pd.read_sql_query(q2,conn)

,facility,usage
0,Massage Room 2,54
1,Squash Court,418
2,Table Tennis,794
3,Pool Table,856
4,Snooker Table,860
5,Tennis Court 2,882
6,Massage Room 1,884
7,Tennis Court 1,957
8,Badminton Court,1086



### /* Q13: Find the facilities usage by month, but not guests */


In [10]:
q3='''
select f.name as facility,strftime('%m',b.starttime) as month,sum(b.slots) as usage
from Bookings as b  inner join Facilities as f on b.facid=f.facid
where b.memid!=0
group by facility,month
order by facility,month
'''
pd.read_sql_query(q3,conn)

,facility,month,usage
0,Badminton Court,07,165
1,Badminton Court,08,414
2,Badminton Court,09,507
3,Massage Room 1,07,166
4,Massage Room 1,08,316
5,Massage Room 1,09,402
6,Massage Room 2,07,8
7,Massage Room 2,08,18
8,Massage Room 2,09,28
9,Pool Table,07,110
